In [1]:
import sys
sys.path.append('../')
import pandas as pd
import joblib
from helper_files.modelling_functions import get_data, process_data, run_optuna_trials, evaluate_models, get_data_config, get_best_model
from helper_files.modelling_configs import *
from helper_files.etv_model import ETVModel

/Users/bradeneberhard/miniforge3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_path = '../data/processed/data_splits_1003.jblb'
train_df, test_df_time, test_df_random, test_df_thrower = get_data(data_path)
test_dfs = [test_df_random, test_df_time, test_df_thrower]
storage = 'sqlite:///model_tuning.db'
train_df = train_df[train_df['throw_distance'] > 3]

In [3]:
# features = thrower_context
# target = 'outcome' 
# data_config = get_data_config(features, target, train_df, test_dfs, False)
# best_models, best_params = run_optuna_trials(data_config, model_config, suffix='fv_base', delete_models=True, storage=storage)
# metrics_df = evaluate_models(best_models, data_config)
# metrics_df

[I 2024-10-04 22:55:58,238] A new study created in RDB with name: logreg_fv_base
[I 2024-10-04 22:55:59,635] Trial 0 finished with value: 0.5910446390382412 and parameters: {}. Best is trial 0 with value: 0.5910446390382412.
[I 2024-10-04 22:55:59,856] A new study created in RDB with name: xgb_fv_base
[I 2024-10-04 22:56:00,576] Trial 0 finished with value: 0.594004312035293 and parameters: {'learning_rate': 0.13107477827645117, 'max_depth': 6, 'n_estimators': 18}. Best is trial 0 with value: 0.594004312035293.
[I 2024-10-04 22:56:02,868] Trial 1 finished with value: 0.5673123214950625 and parameters: {'learning_rate': 0.5322814467473493, 'max_depth': 8, 'n_estimators': 110}. Best is trial 0 with value: 0.594004312035293.
[I 2024-10-04 22:56:04,792] Trial 2 finished with value: 0.5873292119088667 and parameters: {'learning_rate': 0.0031116920775324805, 'max_depth': 10, 'n_estimators': 53}. Best is trial 0 with value: 0.594004312035293.
[I 2024-10-04 22:56:05,193] Trial 3 finished with 

,Model,train_AUC,train_Accuracy,train_NPV,test_df_random_AUC,test_df_random_Accuracy,test_df_random_NPV,test_df_time_AUC,test_df_time_Accuracy,test_df_time_NPV,test_df_thrower_AUC,test_df_thrower_Accuracy,test_df_thrower_NPV
0,logreg,0.591053,0.729723,0.000000,0.591746,0.721298,0.000000,0.583378,0.735662,0.000000,0.581403,0.742189,0.000000
1,xgb,0.597701,0.729810,0.530562,0.596609,0.721631,0.613445,0.591722,0.735279,0.348837,0.587390,0.741923,0.387097


In [4]:
features = thrower_context + game_context
target = 'point_outcome'  
data_config = get_data_config(features, target, train_df, test_dfs, False)
best_models, best_params = run_optuna_trials(data_config, model_config, suffix='fv_point_game', delete_models=True, storage=storage)
metrics_df = evaluate_models(best_models, data_config)
metrics_df

[I 2024-10-04 22:56:29,558] A new study created in RDB with name: logreg_fv_point_game
[I 2024-10-04 22:56:30,696] Trial 0 finished with value: 0.6413188702556836 and parameters: {}. Best is trial 0 with value: 0.6413188702556836.
[I 2024-10-04 22:56:30,960] A new study created in RDB with name: xgb_fv_point_game
[I 2024-10-04 22:56:35,270] Trial 0 finished with value: 0.6589073832252555 and parameters: {'learning_rate': 0.0023585748716443596, 'max_depth': 9, 'n_estimators': 112}. Best is trial 0 with value: 0.6589073832252555.
[I 2024-10-04 22:56:39,948] Trial 1 finished with value: 0.6568948868604233 and parameters: {'learning_rate': 0.0027461248169946433, 'max_depth': 5, 'n_estimators': 191}. Best is trial 0 with value: 0.6589073832252555.
[I 2024-10-04 22:56:41,199] Trial 2 finished with value: 0.6551397516855098 and parameters: {'learning_rate': 0.001826755953014974, 'max_depth': 6, 'n_estimators': 33}. Best is trial 0 with value: 0.6589073832252555.
[I 2024-10-04 22:56:43,765] Tr

,Model,train_AUC,train_Accuracy,train_NPV,test_df_random_AUC,test_df_random_Accuracy,test_df_random_NPV,test_df_time_AUC,test_df_time_Accuracy,test_df_time_NPV,test_df_thrower_AUC,test_df_thrower_Accuracy,test_df_thrower_NPV
0,logreg,0.641728,0.713629,0.565115,0.642119,0.712884,0.571882,0.637963,0.696522,0.549747,0.631667,0.732519,0.556240
1,xgb,0.726264,0.747463,0.795859,0.725885,0.748045,0.812982,0.717323,0.730828,0.785142,0.724255,0.766040,0.804971


In [5]:
features = thrower_context + receiver_context
target = 'completion'  
data_config = get_data_config(features, target, train_df, test_dfs, False)
best_models, best_params = run_optuna_trials(data_config, model_config, suffix='cp_base', delete_models=True, storage=storage)
metrics_df = evaluate_models(best_models, data_config)
metrics_df

[I 2024-10-04 22:57:23,779] A new study created in RDB with name: logreg_cp_base
[I 2024-10-04 22:57:24,600] Trial 0 finished with value: 0.7792972760027772 and parameters: {}. Best is trial 0 with value: 0.7792972760027772.
[I 2024-10-04 22:57:24,857] A new study created in RDB with name: xgb_cp_base
[I 2024-10-04 22:57:28,242] Trial 0 finished with value: 0.7770526074887728 and parameters: {'learning_rate': 0.002939080357610589, 'max_depth': 6, 'n_estimators': 142}. Best is trial 0 with value: 0.7770526074887728.
[I 2024-10-04 22:57:32,038] Trial 1 finished with value: 0.8046333695050132 and parameters: {'learning_rate': 0.04196201274863669, 'max_depth': 7, 'n_estimators': 170}. Best is trial 1 with value: 0.8046333695050132.
[I 2024-10-04 22:57:37,434] Trial 2 finished with value: 0.7784757644131254 and parameters: {'learning_rate': 0.0010049080335679003, 'max_depth': 8, 'n_estimators': 193}. Best is trial 1 with value: 0.8046333695050132.
[I 2024-10-04 22:57:39,034] Trial 3 finishe

,Model,train_AUC,train_Accuracy,train_NPV,test_df_random_AUC,test_df_random_Accuracy,test_df_random_NPV,test_df_time_AUC,test_df_time_Accuracy,test_df_time_NPV,test_df_thrower_AUC,test_df_thrower_Accuracy,test_df_thrower_NPV
0,logreg,0.779201,0.937436,0.440358,0.731751,0.932039,0.451316,0.714179,0.932597,0.424719,0.755498,0.935234,0.451613
1,xgb,0.835615,0.942488,0.724511,0.783514,0.936610,0.744224,0.756744,0.938786,0.744027,0.793899,0.940467,0.709016


In [6]:
features = thrower_context + receiver_context + throw_context + game_context
target = 'completion'
data_config = get_data_config(features, target, train_df, test_dfs, False)
best_models, best_params = run_optuna_trials(data_config, model_config, suffix='cp_game', delete_models=True, storage=storage)
metrics_df = evaluate_models(best_models, data_config)
metrics_df

[I 2024-10-04 22:58:14,781] A new study created in RDB with name: logreg_cp_game
[I 2024-10-04 22:58:17,210] Trial 0 finished with value: 0.7911996717007828 and parameters: {}. Best is trial 0 with value: 0.7911996717007828.
[I 2024-10-04 22:58:18,492] A new study created in RDB with name: xgb_cp_game
[I 2024-10-04 22:58:23,884] Trial 0 finished with value: 0.814303946818605 and parameters: {'learning_rate': 0.05576119096858485, 'max_depth': 5, 'n_estimators': 139}. Best is trial 0 with value: 0.814303946818605.
[I 2024-10-04 22:58:26,022] Trial 1 finished with value: 0.7104799397156002 and parameters: {'learning_rate': 0.002057295879232203, 'max_depth': 1, 'n_estimators': 128}. Best is trial 0 with value: 0.814303946818605.
[I 2024-10-04 22:58:32,313] Trial 2 finished with value: 0.8104997975690532 and parameters: {'learning_rate': 0.17479145034738205, 'max_depth': 7, 'n_estimators': 164}. Best is trial 0 with value: 0.814303946818605.
[I 2024-10-04 22:58:34,324] Trial 3 finished with

,Model,train_AUC,train_Accuracy,train_NPV,test_df_random_AUC,test_df_random_Accuracy,test_df_random_NPV,test_df_time_AUC,test_df_time_Accuracy,test_df_time_NPV,test_df_thrower_AUC,test_df_thrower_Accuracy,test_df_thrower_NPV
0,logreg,0.791204,0.937376,0.443609,0.721586,0.931989,0.453682,0.690462,0.932390,0.421277,0.745416,0.935121,0.450633
1,xgb,0.855423,0.945203,0.835923,0.829988,0.939105,0.833780,0.810398,0.940289,0.808917,0.840585,0.943046,0.860169


In [7]:
# features = thrower_context + receiver_context + throw_context + game_context + player_context
# target = 'completion' 
# data_config = get_data_config(features, target, train_df, test_dfs, False)
# best_models, best_params = run_optuna_trials(data_config, model_config, suffix='cp_game_player', delete_models=True, storage=storage)
# metrics_df = evaluate_models(best_models, data_config)
# metrics_df

In [8]:
study_name='xgb_cp_game'
features = thrower_context + receiver_context + throw_context + game_context
target = 'completion'
cp_model = get_best_model(study_name, storage, train_df, test_dfs, model_config, features, target, False)
study_name='xgb_fv_point_game'
features = thrower_context + game_context
target = 'point_outcome'
fv_model= get_best_model(study_name, storage, train_df, test_dfs, model_config, features, target, False)


etv_model = ETVModel(cp_model, fv_model)
joblib.dump({'cp_model':cp_model, 'fv_model':fv_model, 'etv_model':etv_model}, '../data/models/all_models.jblb')

['../data/models/all_models.jblb']